In [5]:
import psycopg2 as pg
import json

In [2]:
conn = pg.connect("dbname=bart_twitter user=brynstark")
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM bart_data WHERE ")

In [62]:
cur.close()
conn.close()

Taking a look at the BART data from the file:

In [3]:
with open('bart_schedules_by_route.json') as f:
    scheds = f.read().split('\n')

In [4]:
scheds[0]

'{"uri": "http://api.bart.gov/api/sched.aspx?cmd=routesched&route=1", "date": "7/13/2016", "sched_num": "39", "route": {"train": [{"@index": "1", "stop": [{"@station": "PITT", "@origTime": "4:02 AM", "@bikeflag": "1"}, {"@station": "NCON", "@origTime": "4:08 AM", "@bikeflag": "1"}, {"@station": "CONC", "@origTime": "4:12 AM", "@bikeflag": "1"}, {"@station": "PHIL", "@origTime": "4:17 AM", "@bikeflag": "1"}, {"@station": "WCRK", "@origTime": "4:20 AM", "@bikeflag": "1"}, {"@station": "LAFY", "@origTime": "4:25 AM", "@bikeflag": "1"}, {"@station": "ORIN", "@origTime": "4:30 AM", "@bikeflag": "1"}, {"@station": "ROCK", "@origTime": "4:35 AM", "@bikeflag": "1"}, {"@station": "MCAR", "@origTime": "4:38 AM", "@bikeflag": "1"}, {"@station": "19TH", "@origTime": "4:42 AM", "@bikeflag": "1"}, {"@station": "12TH", "@origTime": "4:43 AM", "@bikeflag": "1"}, {"@station": "WOAK", "@origTime": "4:48 AM", "@bikeflag": "1"}, {"@station": "EMBR", "@origTime": "4:55 AM", "@bikeflag": "1"}, {"@station": 

In [12]:
sched0 = json.loads(scheds[0])
sched0.keys()

dict_keys(['uri', 'route', 'sched_num', 'date', 'message'])

In [67]:
# This is a list of the various trains.
print('{0} trains'.format(len(sched0['route']['train'])))
sched0['route']['train'][-1:]

96 trains


[{'@index': '96',
  'stop': [{'@bikeflag': '1', '@origTime': '11:59 PM', '@station': 'PITT'},
   {'@bikeflag': '1', '@origTime': '12:05 AM', '@station': 'NCON'},
   {'@bikeflag': '1', '@origTime': '12:09 AM', '@station': 'CONC'},
   {'@bikeflag': '1', '@origTime': '12:14 AM', '@station': 'PHIL'},
   {'@bikeflag': '1', '@origTime': '12:17 AM', '@station': 'WCRK'},
   {'@bikeflag': '1', '@origTime': '12:22 AM', '@station': 'LAFY'},
   {'@bikeflag': '1', '@origTime': '12:27 AM', '@station': 'ORIN'},
   {'@bikeflag': '1', '@origTime': '12:32 AM', '@station': 'ROCK'},
   {'@bikeflag': '1', '@origTime': '12:46 AM', '@station': 'MCAR'},
   {'@bikeflag': '1', '@origTime': '12:49 AM', '@station': '19TH'},
   {'@bikeflag': '1', '@origTime': '12:51 AM', '@station': '12TH'},
   {'@bikeflag': '1', '@origTime': '12:55 AM', '@station': 'WOAK'},
   {'@bikeflag': '1', '@origTime': '1:02 AM', '@station': 'EMBR'},
   {'@bikeflag': '1', '@origTime': '1:04 AM', '@station': 'MONT'},
   {'@bikeflag': '1', '@

In [21]:
sched0['route']['train'][0]['stop']

[{'@bikeflag': '1', '@origTime': '4:02 AM', '@station': 'PITT'},
 {'@bikeflag': '1', '@origTime': '4:08 AM', '@station': 'NCON'},
 {'@bikeflag': '1', '@origTime': '4:12 AM', '@station': 'CONC'},
 {'@bikeflag': '1', '@origTime': '4:17 AM', '@station': 'PHIL'},
 {'@bikeflag': '1', '@origTime': '4:20 AM', '@station': 'WCRK'},
 {'@bikeflag': '1', '@origTime': '4:25 AM', '@station': 'LAFY'},
 {'@bikeflag': '1', '@origTime': '4:30 AM', '@station': 'ORIN'},
 {'@bikeflag': '1', '@origTime': '4:35 AM', '@station': 'ROCK'},
 {'@bikeflag': '1', '@origTime': '4:38 AM', '@station': 'MCAR'},
 {'@bikeflag': '1', '@origTime': '4:42 AM', '@station': '19TH'},
 {'@bikeflag': '1', '@origTime': '4:43 AM', '@station': '12TH'},
 {'@bikeflag': '1', '@origTime': '4:48 AM', '@station': 'WOAK'},
 {'@bikeflag': '1', '@origTime': '4:55 AM', '@station': 'EMBR'},
 {'@bikeflag': '1', '@origTime': '4:56 AM', '@station': 'MONT'},
 {'@bikeflag': '1', '@origTime': '4:58 AM', '@station': 'POWL'},
 {'@bikeflag': '1', '@ori

It will be most useful to join the two data sources, tweets and route schedules, based on time and if the content of the tweet concerns that particular BART train.

I'll explore how to navigate to the time of day within our Twitter JSON objects.

In [22]:
with open('test_tweets/bart train.json') as f:
    tweets = f.read().split('\n')

In [23]:
tweets[0]

'{"created_at":"Mon Jul 11 02:19:47 +0000 2016","id":752326490189168640,"id_str":"752326490189168640","text":"RT @MTL_NSMITH: @OrganikHipHop tough decision.... I hope you keep it going.... KOTD is excellent and you guys have done a great job","source":"\\u003ca href=\\"http:\\/\\/twitter.com\\/download\\/android\\" rel=\\"nofollow\\"\\u003eTwitter for Android\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":719727925617160192,"id_str":"719727925617160192","name":"Christian","screen_name":"Christi82339229","location":null,"url":null,"description":"first time on this Social Media shit so bear with me here,  christianelias040@gmail.com, Soundcloud: finest23\\n - Student of CHS 18 hoe","protected":false,"verified":false,"followers_count":275,"friends_count":263,"listed_count":3,"favourites_count":2187,"statuses_count":1942,"created_at":"Tue

In [24]:
tweet0 = json.loads(tweets[0])
tweet0.keys()

dict_keys(['timestamp_ms', 'truncated', 'geo', 'text', 'retweeted', 'in_reply_to_status_id_str', 'created_at', 'lang', 'contributors', 'filter_level', 'favorite_count', 'id_str', 'in_reply_to_user_id_str', 'id', 'in_reply_to_user_id', 'retweet_count', 'coordinates', 'user', 'in_reply_to_status_id', 'is_quote_status', 'entities', 'retweeted_status', 'favorited', 'source', 'in_reply_to_screen_name', 'place'])

In [25]:
tweet0['created_at']

'Mon Jul 11 02:19:47 +0000 2016'

In [29]:
time0 = tweet0['created_at'].split( )
time0[3]  # the time as a string

'02:19:47'

In [63]:
from datetime import datetime

# Example obtained from Stack Overflow:
# date_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

# Applying that to our tweet to convert the string to datetime:
date_object = datetime.strptime('02:19:47', '%I:%M:%S')

In [64]:
# Without the date included, it stores the date as Jan 1, 1900 by default.
date_object

datetime.datetime(1900, 1, 1, 2, 19, 47)

In [37]:
from datetime import time

In [48]:
# Just the time:
datetime.time(date_object)

datetime.time(2, 19, 47)

Can we get the BART schedule time in the same format so that we'd then be able to join them?

In [58]:
# Walking the JSON to the time of a specific stop
b_time0 = sched0['route']['train'][0]['stop'][0]['@origTime']
b_time0

'4:02 AM'

In [65]:
# Converting the string to a datetime object
date_object = datetime.strptime('4:02 AM', '%I:%M %p')

In [66]:
date_object

datetime.datetime(1900, 1, 1, 4, 2)

In [60]:
# Converting it to a universal format
datetime.time(date_object)

datetime.time(4, 2)

In [61]:
# What would this look like with a time that's pm?
date_object = datetime.strptime('4:02 PM', '%I:%M %p')
datetime.time(date_object)

datetime.time(16, 2)

In [ ]:
# ^ Military time. Perfecto.

Perhaps as a first go, joining on location/station would be better than time. That way, a user could see all tweets surroundning a certain station or area, and look at the BART schedule corresponding. In future versions of this app, the tweets and BART schedules would be streamed and processed in real time anyways, so in that regard it may be better to simply return the most recent tweets surrounding a station/area.